In [1]:
!pip install transformers
!pip install rouge_score

     |████████████████████████████████| 2.9 MB 11.3 MB/s 
     |████████████████████████████████| 895 kB 33.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.2 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 636 kB 37.8 MB/s 
     |████████████████████████████████| 109 kB 51.9 MB/s 
     |████████████████████████████████| 546 kB 40.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    tokenizer="pierreguillou/bert-base-cased-squad-v1.1-portuguese"
)

predictions = qa_pipeline({
    'context': "O rio Nilo é o rio mais comprido do mundo",
    'question': "Qual é o rio mais comprido do mundo?"
})

print(predictions)

Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

{'score': 0.8113061785697937, 'start': 0, 'end': 10, 'answer': 'O rio Nilo'}


In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [5]:
%cd /content/drive/Shareddrives/Explainable QA/Base de dados/QAs
%ls

/content/drive/Shareddrives/Explainable QA/Base de dados/QAs
 1109950027.csv   flavio_manual_generation_pt.xlsx
 1676173966.csv   flavio_manual_generation.xlsx
 596840551.csv    sample_to_interpret.csv
 936823110.csv   'Sem context'/
 Automatic/


In [6]:
import pandas as pd
df = pd.read_excel('flavio_manual_generation_pt.xlsx')
df.head()

,#,Source,Passage,Question,Answer
0,1,Campos Basin.txt,A Bacia de Campos é uma das 12 bacias sediment...,Quantas bacias sedimentares costeiras existem ...,12.
1,2,Campos Basin.txt,A bacia é nomeada após a cidade de Campos dos ...,Por que a Bacia de Campos é chamada?,Depois da cidade de Campos dos Goytacazes.
2,3,Campos Basin.txt,A Bacia de Campos é uma das 12 bacias sediment...,Onde está localizado a parte onshore da bacia ...,Está localizado perto do Rio de Janeiro.
3,4,Campos Basin.txt,A Bacia de Campos é uma das 12 bacias sediment...,Quão grande é a bacia de Campos?,São cerca de 115.000 quilômetros quadrados.
4,5,Campos Basin.txt,A Bacia de Campos é uma das 12 bacias sediment...,Quantos anos tem a bacia de Campos?,A bacia é de 145 milhões de velha.


In [7]:
source = df.values.tolist()

In [8]:
import time
start_time = time.clock()

true_answers = []
gen_answers = []
passages = []
questions =[]
for i in range(len(source)):
    print(i)
    predictions = qa_pipeline({
    'context': source[i][2],
    'question': source[i][3]
    })
    # passages.append(source[i][2])
    # questions.append(source[i][3])
    # gen_answers.append(predictions["answer"])
    # true_answers.append(source[i][4])
print(time.clock() - start_time, "seconds")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
47.575970000000005 seconds


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [9]:
47/90

0.5222222222222223

In [10]:
!nvidia-smi

Mon Oct  4 23:10:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from rouge_score import rouge_scorer
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
from nltk.metrics import f_measure
import nltk
import string
import re
import collections

def get_tokens(sentence):
    tokens = word_tokenize(sentence)
    return tokens

def normalize_text(sentence):

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    return white_space_fix(remove_punc(lower(sentence)))

def f1_score_qa(text_true,text_pred):
    "squad"
    text_true = normalize_text(text_true)
    text_pred = normalize_text(text_pred)
    true_tokens = get_tokens(text_true)
    pred_tokens = get_tokens(text_pred)
    common = collections.Counter(true_tokens) & collections.Counter(pred_tokens)
    num_same = sum(common.values())
    if len(true_tokens) == 0 or len(pred_tokens) == 0:
        return int(true_tokens==pred_tokens),int(true_tokens==pred_tokens),int(true_tokens==pred_tokens)
    if num_same == 0:
        return 0,0,0
    precision = 1 * num_same / len(pred_tokens)
    recall = 1 * num_same / len(true_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

def em_qa(text_true,text_pred):
    return int(normalize_text(text_true) == normalize_text(text_pred))

def rouge_l_qa(text_true,text_pred):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(text_true,text_pred)

    return scores["rougeL"].fmeasure

def em_qa_overall(text_true,text_pred):
    scores = 0
    for i in range(len(text_true)):
        scores += em_qa(text_true[i],text_pred[i])
    score = scores/len(text_true)
    return score

def f1_score_qa_overall(text_true,text_pred):
    f = 0
    for i in range(len(text_true)):
        fi,pi,ri = f1_score_qa(text_true[i],text_pred[i])
        f+=fi
    f = f/len(text_true)
    return f

def rouge_l_qa_overall(text_true,text_pred):
    scores = 0
    for i in range(len(text_true)):
        score = rouge_l_qa(text_true[i],text_pred[i])
        scores += score
    scores = scores/len(text_true)
    return scores

def scores(func,text_true,text_pred):
    if func=='rouge_l_qa':
        score_value = rouge_l_qa(text_true,text_pred)
    if func=='em_qa':
        score_value = em_qa(text_true,text_pred)
    if func=='f1_score_qa':
        score_value = f1_score_qa(text_true,text_pred)
    return (func, score_value)
        

def overall(func,text_true,text_pred):
    if func=='rouge_l_qa_overall':
        score_value = rouge_l_qa_overall(text_true,text_pred)
    if func=='em_qa_overall':
        score_value = em_qa_overall(text_true,text_pred)
    if func=='f1_score_qa_overall':
        score_value = f1_score_qa_overall(text_true,text_pred)
    return (func, score_value)

In [ ]:
print("f1-score: " + str(overall("f1_score_qa_overall", true_answers, gen_answers)))
print("EM: " + str(overall('em_qa_overall', true_answers, gen_answers)))
print("Rouge-L: " + str(overall('rouge_l_qa_overall', true_answers, gen_answers)))

f1-score: ('f1_score_qa_overall', 0.47020767729265134)
EM: ('em_qa_overall', 0.1)
Rouge-L: ('rouge_l_qa_overall', 0.4843389687790932)


In [ ]:
df = pd.DataFrame({'question':questions, 'passages': passages, 'true_answers': true_answers, "gen_answers": gen_answers})



In [ ]:
from tqdm import tqdm
model_name = "test"
for _, row in tqdm(df.iterrows()):
    # F1-score
    _, f1_score = scores('f1_score_qa', row['gen_answers'], row['true_answers'])
    df.loc[df['question']==row['question'], '{}_f1'.format(model_name)] = round(f1_score[0], 2)

    # Rouge
    _, rouge_l = scores('rouge_l_qa', row['gen_answers'], row['true_answers'])
    rouge_l_f = rouge_l
    df.loc[df['question']==row['question'], '{}_rouge_l_f'.format(model_name)] = round(rouge_l_f, 2)

    # Exact Match
    _, em = scores('em_qa', row['gen_answers'], row['true_answers'])
    df.loc[df['question']==row['question'], '{}_em'.format(model_name)] = round(em, 2)

90it [00:00, 356.11it/s]


In [ ]:
df

,question,passages,true_answers,gen_answers,test_f1,test_rouge_l_f,test_em
0,Quantas bacias sedimentares costeiras existem ...,A Bacia de Campos é uma das 12 bacias sediment...,12.,12,1.00,1.00,1.0
1,Por que a Bacia de Campos é chamada?,A bacia é nomeada após a cidade de Campos dos ...,Depois da cidade de Campos dos Goytacazes.,após a cidade de Campos dos Goytacazes,0.71,0.67,0.0
2,Onde está localizado a parte onshore da bacia ...,A Bacia de Campos é uma das 12 bacias sediment...,Está localizado perto do Rio de Janeiro.,perto do Rio de Janeiro,0.83,0.83,0.0
3,Quão grande é a bacia de Campos?,A Bacia de Campos é uma das 12 bacias sediment...,São cerca de 115.000 quilômetros quadrados.,115.000 quilômetros quadrados,0.67,0.71,0.0
4,Quantos anos tem a bacia de Campos?,A Bacia de Campos é uma das 12 bacias sediment...,A bacia é de 145 milhões de velha.,145-130 milhões de anos atrás,0.31,0.50,0.0
...,...,...,...,...,...,...,...
85,O que o Porto Recife é usado para?,Este porto internacional atende ao RMR e possu...,Este porto internacional atende ao RMR e possu...,cruzeiros e cargas,0.33,0.33,0.0
86,Onde está localizado o porto de Recife?,"O Porto Recife, está localizado em Recife Anti...","O Porto Recife, está localizado em Recife Anti...",Recife Antigo,0.14,0.14,0.0
87,O que torna o Recife Porto diferente de outras...,Este porto internacional atende ao RMR e possu...,A porta se diferencia de outras portas localiz...,a porta não tem nenhuma interferência com o ce...,0.53,0.58,0.0
88,Qual é o acesso ao porto?,"Existem dois canais de acesso à porta, ambas a...",Canal do Sul e Canal Norte.,canal sul,0.50,0.50,0.0


In [ ]:
df.loc[df['test_f1']>=0.7,:]

,question,passages,true_answers,gen_answers,test_f1,test_rouge_l_f,test_em
0,Quantas bacias sedimentares costeiras existem ...,A Bacia de Campos é uma das 12 bacias sediment...,12.,12,1.00,1.00,1.0
1,Por que a Bacia de Campos é chamada?,A bacia é nomeada após a cidade de Campos dos ...,Depois da cidade de Campos dos Goytacazes.,após a cidade de Campos dos Goytacazes,0.71,0.67,0.0
2,Onde está localizado a parte onshore da bacia ...,A Bacia de Campos é uma das 12 bacias sediment...,Está localizado perto do Rio de Janeiro.,perto do Rio de Janeiro,0.83,0.83,0.0
6,Qual é o significado da expressão brasileira p...,"Uma famosa expressão no Brasil é ""de Oiapoque ...","Significa ""do extremo sul até o extremo norte""...",do extremo sul até o extremo norte do país,0.95,0.95,0.0
11,Quais são as praias de Amapá?,O litoral de Amapá é quase 600 km de comprimen...,"Existem apenas três praias: Fazendinha, Boca d...","Fazendinha, Boca do Inferno e Goiabal",0.75,0.71,0.0
23,"Até 2017, qual foi a participação do petróleo ...",Uma das maiores bacias sedimentares brasileira...,35%.,35%,1.00,1.00,1.0
25,Por que a Bacia de Santos tem esse nome?,A Bacia de Santos é nomeada após a cidade cost...,A Bacia de Santos é nomeada após a cidade cost...,a cidade costeira de Santos no estado de São P...,0.74,0.76,0.0
30,Qual é o porto líder do Brasil no tráfego de c...,O Porto de Santos (em Português: Porto de Sant...,O porto de Santos.,Porto de Santos,0.86,0.86,0.0
31,Onde está localizado o porto de Santos?,O Porto de Santos (em Português: Porto de Sant...,"Ele está localizado na cidade de Santos, estad...","cidade de Santos, estado de São Paulo, Brasil",0.80,0.82,0.0
36,Quando o boom da urbanização do Porto de Santo...,"No início do século XX, maior revisão e urbani...",No início do século XX.,início do século XX,0.89,0.92,0.0


In [ ]:
sample = df.loc[df['test_f1']>=0.7,:]

In [ ]:
sample.to_csv("sample_to_interpret.csv",index=False)